In [59]:
import ethnicolr as ethc
import pandas as pd
import numpy as np
import sklearn.metrics as met

In [21]:
cs = pd.read_excel("~/Desktop/census_2010.xlsx")

In [22]:
# Drop unimportant columns/The "ALL OTHER NAMES" row
cs = cs.drop(columns = ["rank", "count", "prop100k", "cum_prop100k", "pctaian", "pct2prace"])
cs = cs.drop([162253])

In [24]:
# Replaced hidden values with 0's
cs = cs.replace({"(S)": 0})

In [40]:
def race_actual(row):
    race_id = np.argmax([row['pctwhite'],row['pctblack'],row['pctapi'],row['pcthispanic']])
    race_id_to_name = {0: "nh_white", 1: "nh_black", 2: "asian", 3: "hispanic"}
    race_name = race_id_to_name[race_id]
    return race_name
cs["race_actual"] = cs.apply(race_actual, axis = 1)

In [41]:
output = ethc.pred_fl_reg_ln(cs, "name")

In [48]:
race_actual_pred = output.drop(columns = ["pctwhite", "pctblack", "pctapi", "pcthispanic", "nh_white", "asian", "nh_black", "hispanic"])
race_actual_pred = race_actual_pred.rename(columns = {"race": "race_pred"})

In [56]:
race_actual_pred = race_actual_pred.fillna('None')

array(['nh_white', 'nh_black', 'hispanic', 'asian', 'None'], dtype=object)

In [65]:
y_true = race_actual_pred["race_actual"]
y_pred = race_actual_pred["race_pred"]
labels = ["nh_white", "nh_black", "asian", "hispanic", "None"]

In [66]:
met.confusion_matrix(y_true, y_pred, labels = labels)

array([[130905,    882,    487,   1466,      1],
       [  6617,    984,    151,    105,      0],
       [  6255,    116,   2219,    769,      0],
       [  5560,     83,     69,   5584,      0],
       [     0,      0,      0,      0,      0]])

In [73]:
print(met.accuracy_score(y_true, y_pred))

0.8609517235428621


In [74]:
print(met.precision_score(y_true, y_pred, average=None, labels = labels))
print(met.precision_score(y_true, y_pred, average='weighted', labels = labels))

[0.87657446 0.47651332 0.75837321 0.7046946  0.        ]
0.8384175016068367


In [75]:
print(met.recall_score(y_true, y_pred, average=None, labels = labels))
print(met.recall_score(y_true, y_pred, average='weighted', labels = labels))

[0.97879483 0.12523864 0.23709798 0.49433428 0.        ]
0.8609517235428621


In [76]:
print(met.f1_score(y_true, y_pred, average=None, labels = labels))
print(met.f1_score(y_true, y_pred, average='weighted', labels = labels))

[0.92486876 0.19834711 0.36125356 0.58106139 0.        ]
0.8332414697509095
